In [23]:
import torch
from sklearn.calibration import CalibratedClassifierCV
import torch.optim as optim
import torch.nn as nn
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# Get Data and Base Model

In [31]:
dataset_name = 'CUB'
concept_name = 'has_back_color::black'

metadata_df = pd.read_csv(f'../Data/{dataset_name}/metadata.csv')
embeddings = torch.load(f'Embeddings/{dataset_name}/embeddings.pt') 

train_indices = metadata_df[metadata_df['split'] == 'train'].index.tolist()
test_indices = metadata_df[metadata_df['split'] == 'test'].index.tolist()
calibration_indices = metadata_df[metadata_df['split'] == 'calibration'].index.tolist()

X_train = embeddings[train_indices].numpy()
X_test = embeddings[test_indices].numpy()
X_cal = embeddings[calibration_indices].numpy()

y_train = np.array(metadata_df[(metadata_df['split'] == 'train')][concept_name])
y_test = np.array(metadata_df[(metadata_df['split'] == 'test')][concept_name])
y_cal = np.array(metadata_df[(metadata_df['split'] == 'calibration')][concept_name])

In [7]:
# for now, train a basic classifier

In [29]:
base_model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=200, random_state=42)
base_model.fit(X_train, y_train)
y_pred = base_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 74.69%


## Platt Scaling

In [34]:
platt_calibrated_model = CalibratedClassifierCV(base_model, method='sigmoid', cv='prefit')
platt_calibrated_model.fit(X_cal, y_cal)  # Fit calibration on validation set

y_pred = platt_calibrated_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 75.70%


## Isotonic Regression

In [35]:
isotonic_calibrated_model = CalibratedClassifierCV(base_model, method='isotonic', cv='prefit') 
isotonic_calibrated_model.fit(X_cal, y_cal)

y_pred = isotonic_calibrated_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 75.65%


## Temperature Scaling

In [42]:
class TemperatureScaling(nn.Module):
    """A simple module for temperature scaling."""
    def __init__(self, base_model):
        super(TemperatureScaling, self).__init__()
        self.temperature = nn.Parameter(torch.ones(1) * 1.0)  # Initial temperature is set to 1.0
        self.base_model = base_model

    #def forward(self, x, return_logits=True):
    def forward(self, x):
        x = x.to(device)
        #logits = self.base_model(x, return_logits=True)
        logits = self.base_model(x)
        scaled_logits = logits / self.temperature
        if return_logits:
            return scaled_logits
        else:
            return F.softmax(calibrated_logits, dim=1)

def train_temperature_scaling(base_model, X_cal, y_cal):
    """Train temperature scaling using negative log-likelihood. """
    
    # Initialize the temperature scaling model and move it to the appropriate device
    temperature_model = TemperatureScaling(base_model).to(device)
    optimizer = optim.LBFGS([temperature_model.temperature], max_iter=50, line_search_fn="strong_wolfe")
    
    # Move inputs and labels to the specified device
    X_cal = torch.FloatTensor(X_cal).to(device)
    y_cal = torch.LongTensor(y_cal).to(device)
    
    # Use cross-entropy as the loss function
    criterion = nn.CrossEntropyLoss()
    
    # Closure function for the optimizer
    def closure():
        optimizer.zero_grad()  # Clear gradients
        scaled_logits = temperature_model(X_cal)
        #scaled_logits = temperature_model(X_cal, return_logits=True)  # Scale the logits using the current temperature
        loss = criterion(scaled_logits, y_cal)  # Calculate cross-entropy loss
        loss.backward()  # Backpropagate
        return loss
    
    # Perform optimization step
    optimizer.step(closure)
    
    # Print the optimal temperature value
    print(f"Optimal temperature: {temperature_model.temperature.item():.4f}")
    
    return temperature_model

device = 'cpu' #just for sklearn
temperature_calibrated_model = train_temperature_scaling(base_model, X_cal, y_cal)

y_pred = temperature_calibrated_model(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.